# Importing libraries, Examining data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [2]:
# Read csv
originaldata = pd.read_csv('https://raw.githubusercontent.com/Bomsk/DA3_2023/main/Assignment%202/listings.csv')

# Check dataset
originaldata

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,197677,https://www.airbnb.com/rooms/197677,20231227173752,2023-12-28,city scrape,Rental unit in Sumida · ★4.77 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,...,4.83,4.54,4.79,M130003350,f,1,1,0,0,1.16
1,776070,https://www.airbnb.com/rooms/776070,20231227173752,2023-12-28,city scrape,Home in Kita-ku · ★4.98 · 1 bedroom · 1 bed · ...,NaN,We love Nishinippori because is nearer to Toky...,https://a0.muscache.com/pictures/efd9f039-dbd2...,801494,...,4.98,4.84,4.91,M130000243,f,1,0,1,0,1.85
2,905944,https://www.airbnb.com/rooms/905944,20231227173752,2023-12-28,city scrape,Rental unit in Shibuya · ★4.78 · 2 bedrooms · ...,NaN,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,...,4.91,4.78,4.78,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...,t,5,5,0,0,1.52
3,1016831,https://www.airbnb.com/rooms/1016831,20231227173752,2023-12-28,previous scrape,Home in Setagaya · ★4.95 · 1 bedroom · 2 beds ...,NaN,The location is walkable distance to famous Sh...,https://a0.muscache.com/pictures/airflow/Hosti...,5596383,...,4.98,4.92,4.90,M130001107,f,1,0,1,0,1.99
4,3838856,https://www.airbnb.com/rooms/3838856,20231227173752,2023-12-28,city scrape,Home in Shinjuku · ★4.89 · 3 bedrooms · 5 beds...,NaN,The house is in a quiet and safe street. The s...,https://a0.muscache.com/pictures/miso/Hosting-...,4626879,...,4.95,4.83,4.83,M130018884,f,6,6,0,0,1.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13720,1052613009241026699,https://www.airbnb.com/rooms/1052613009241026699,20231227173752,2023-12-28,city scrape,Rental unit in Shinjuku City · ★New · 3 bedroo...,NaN,★概要★<br />サブカルチャーで有名な秋葉原周辺には大変魅力的な場所がたくさんあります。...,https://a0.muscache.com/pictures/miso/Hosting-...,118634275,...,NaN,NaN,NaN,M130010532,f,2,2,0,0,NaN
13721,1052618956131641134,https://www.airbnb.com/rooms/1052618956131641134,20231227173752,2023-12-28,city scrape,Rental unit in Taito City · ★New · 2 bedrooms ...,NaN,Ichiyu Tokyo Apartment is near Iriya and Asaku...,https://a0.muscache.com/pictures/miso/Hosting-...,292115979,...,NaN,NaN,NaN,M130019957,f,47,45,2,0,NaN
13722,1052621352644395645,https://www.airbnb.com/rooms/1052621352644395645,20231227173752,2023-12-28,city scrape,Rental unit in Shinjuku City · ★New · 2 bedroo...,NaN,新宿駅直通のバス停が徒歩2分の距離にあり、とてもアクセスしやすいです。バスストップから新宿駅...,https://a0.muscache.com/pictures/c6c689cd-2cd4...,320838422,...,NaN,NaN,NaN,M130032122,t,3,3,0,0,NaN
13723,1052624489140939071,https://www.airbnb.com/rooms/1052624489140939071,20231227173752,2023-12-28,city scrape,Rental unit in Shinjuku City · ★New · 3 bedroo...,NaN,【新宿へようこそ！】<br />新宿は20世紀に大幅に開発され、今や東京の代名詞的な存在とな...,https://a0.muscache.com/pictures/miso/Hosting-...,320838422,...,NaN,NaN,NaN,M130022952,t,3,3,0,0,NaN


In [3]:
# Drop rows with missing prices
data = originaldata.dropna(subset=['price'])

data

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,197677,https://www.airbnb.com/rooms/197677,20231227173752,2023-12-28,city scrape,Rental unit in Sumida · ★4.77 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,...,4.83,4.54,4.79,M130003350,f,1,1,0,0,1.16
1,776070,https://www.airbnb.com/rooms/776070,20231227173752,2023-12-28,city scrape,Home in Kita-ku · ★4.98 · 1 bedroom · 1 bed · ...,NaN,We love Nishinippori because is nearer to Toky...,https://a0.muscache.com/pictures/efd9f039-dbd2...,801494,...,4.98,4.84,4.91,M130000243,f,1,0,1,0,1.85
2,905944,https://www.airbnb.com/rooms/905944,20231227173752,2023-12-28,city scrape,Rental unit in Shibuya · ★4.78 · 2 bedrooms · ...,NaN,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,...,4.91,4.78,4.78,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...,t,5,5,0,0,1.52
4,3838856,https://www.airbnb.com/rooms/3838856,20231227173752,2023-12-28,city scrape,Home in Shinjuku · ★4.89 · 3 bedrooms · 5 beds...,NaN,The house is in a quiet and safe street. The s...,https://a0.muscache.com/pictures/miso/Hosting-...,4626879,...,4.95,4.83,4.83,M130018884,f,6,6,0,0,1.95
5,1196177,https://www.airbnb.com/rooms/1196177,20231227173752,2023-12-28,city scrape,Home in 足立区 · ★4.75 · 1 bedroom · 1.5 shared b...,NaN,There are shopping mall near Senjuohashi stati...,https://a0.muscache.com/pictures/72890882/05ec...,5686404,...,4.90,4.70,4.78,M130007760,f,1,0,1,0,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13720,1052613009241026699,https://www.airbnb.com/rooms/1052613009241026699,20231227173752,2023-12-28,city scrape,Rental unit in Shinjuku City · ★New · 3 bedroo...,NaN,★概要★<br />サブカルチャーで有名な秋葉原周辺には大変魅力的な場所がたくさんあります。...,https://a0.muscache.com/pictures/miso/Hosting-...,118634275,...,NaN,NaN,NaN,M130010532,f,2,2,0,0,NaN
13721,1052618956131641134,https://www.airbnb.com/rooms/1052618956131641134,20231227173752,2023-12-28,city scrape,Rental unit in Taito City · ★New · 2 bedrooms ...,NaN,Ichiyu Tokyo Apartment is near Iriya and Asaku...,https://a0.muscache.com/pictures/miso/Hosting-...,292115979,...,NaN,NaN,NaN,M130019957,f,47,45,2,0,NaN
13722,1052621352644395645,https://www.airbnb.com/rooms/1052621352644395645,20231227173752,2023-12-28,city scrape,Rental unit in Shinjuku City · ★New · 2 bedroo...,NaN,新宿駅直通のバス停が徒歩2分の距離にあり、とてもアクセスしやすいです。バスストップから新宿駅...,https://a0.muscache.com/pictures/c6c689cd-2cd4...,320838422,...,NaN,NaN,NaN,M130032122,t,3,3,0,0,NaN
13723,1052624489140939071,https://www.airbnb.com/rooms/1052624489140939071,20231227173752,2023-12-28,city scrape,Rental unit in Shinjuku City · ★New · 3 bedroo...,NaN,【新宿へようこそ！】<br />新宿は20世紀に大幅に開発され、今や東京の代名詞的な存在とな...,https://a0.muscache.com/pictures/miso/Hosting-...,320838422,...,NaN,NaN,NaN,M130022952,t,3,3,0,0,NaN


In [4]:
data.price

0        $12,000.00
1         $8,541.00
2        $27,759.00
4        $26,775.00
5         $6,014.00
            ...    
13720    $80,000.00
13721    $15,000.00
13722    $32,500.00
13723    $56,200.00
13724    $56,000.00
Name: price, Length: 13312, dtype: object

In [5]:
# Clean price column
data["price"] = data["price"].replace('[\$,]', '', regex=True).astype(float)

data.price

0        12000.0
1         8541.0
2        27759.0
4        26775.0
5         6014.0
          ...   
13720    80000.0
13721    15000.0
13722    32500.0
13723    56200.0
13724    56000.0
Name: price, Length: 13312, dtype: float64

In [6]:
# Filter 'accommodates' for 2-6 people
data = data[(data['accommodates'] >= 2) & 
            (data['accommodates'] <= 6)]

data

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,197677,https://www.airbnb.com/rooms/197677,20231227173752,2023-12-28,city scrape,Rental unit in Sumida · ★4.77 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,...,4.83,4.54,4.79,M130003350,f,1,1,0,0,1.16
2,905944,https://www.airbnb.com/rooms/905944,20231227173752,2023-12-28,city scrape,Rental unit in Shibuya · ★4.78 · 2 bedrooms · ...,NaN,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,...,4.91,4.78,4.78,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...,t,5,5,0,0,1.52
5,1196177,https://www.airbnb.com/rooms/1196177,20231227173752,2023-12-28,city scrape,Home in 足立区 · ★4.75 · 1 bedroom · 1.5 shared b...,NaN,There are shopping mall near Senjuohashi stati...,https://a0.muscache.com/pictures/72890882/05ec...,5686404,...,4.90,4.70,4.78,M130007760,f,1,0,1,0,0.85
6,3845889,https://www.airbnb.com/rooms/3845889,20231227173752,2023-12-27,city scrape,Rental unit in Suginami-ku · ★4.86 · 1 bedroom...,NaN,Koenji town is very unique and stimulating pla...,https://a0.muscache.com/pictures/e498a703-e523...,19089769,...,4.96,4.90,4.87,M130000777,f,2,2,0,0,0.97
7,3846001,https://www.airbnb.com/rooms/3846001,20231227173752,2023-12-27,city scrape,Rental unit in Suginami-ku · ★4.92 · Studio · ...,NaN,NaN,https://a0.muscache.com/pictures/d697e82d-cb99...,19089769,...,4.95,4.90,4.85,M130008732,f,2,2,0,0,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13717,1052523990478131414,https://www.airbnb.com/rooms/1052523990478131414,20231227173752,2023-12-28,city scrape,Rental unit in Katsushika City · ★New · 2 bedr...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,552216877,...,NaN,NaN,NaN,M130035708,t,2,2,0,0,NaN
13718,1052570322244321995,https://www.airbnb.com/rooms/1052570322244321995,20231227173752,2023-12-28,city scrape,Home in Shinagawa City · ★New · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,28621978,...,NaN,NaN,NaN,M130025425,f,7,0,7,0,NaN
13719,1052573121521757761,https://www.airbnb.com/rooms/1052573121521757761,20231227173752,2023-12-28,city scrape,Home in Shinagawa City · ★New · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,28621978,...,NaN,NaN,NaN,M130025425,f,7,0,7,0,NaN
13721,1052618956131641134,https://www.airbnb.com/rooms/1052618956131641134,20231227173752,2023-12-28,city scrape,Rental unit in Taito City · ★New · 2 bedrooms ...,NaN,Ichiyu Tokyo Apartment is near Iriya and Asaku...,https://a0.muscache.com/pictures/miso/Hosting-...,292115979,...,NaN,NaN,NaN,M130019957,f,47,45,2,0,NaN


In [7]:
data.room_type.value_counts()

room_type
Entire home/apt    7931
Private room       2276
Hotel room          108
Shared room          61
Name: count, dtype: int64

In [8]:
# filtering out 'Hotel room' & 'Shared room'
data = data[(data['accommodates'] >= 2) & 
            (data['accommodates'] <= 6) &
            (data['room_type'] != 'Hotel room') &
            (data['room_type'] != 'Shared room')]

data

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,197677,https://www.airbnb.com/rooms/197677,20231227173752,2023-12-28,city scrape,Rental unit in Sumida · ★4.77 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,...,4.83,4.54,4.79,M130003350,f,1,1,0,0,1.16
2,905944,https://www.airbnb.com/rooms/905944,20231227173752,2023-12-28,city scrape,Rental unit in Shibuya · ★4.78 · 2 bedrooms · ...,NaN,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,...,4.91,4.78,4.78,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...,t,5,5,0,0,1.52
5,1196177,https://www.airbnb.com/rooms/1196177,20231227173752,2023-12-28,city scrape,Home in 足立区 · ★4.75 · 1 bedroom · 1.5 shared b...,NaN,There are shopping mall near Senjuohashi stati...,https://a0.muscache.com/pictures/72890882/05ec...,5686404,...,4.90,4.70,4.78,M130007760,f,1,0,1,0,0.85
6,3845889,https://www.airbnb.com/rooms/3845889,20231227173752,2023-12-27,city scrape,Rental unit in Suginami-ku · ★4.86 · 1 bedroom...,NaN,Koenji town is very unique and stimulating pla...,https://a0.muscache.com/pictures/e498a703-e523...,19089769,...,4.96,4.90,4.87,M130000777,f,2,2,0,0,0.97
7,3846001,https://www.airbnb.com/rooms/3846001,20231227173752,2023-12-27,city scrape,Rental unit in Suginami-ku · ★4.92 · Studio · ...,NaN,NaN,https://a0.muscache.com/pictures/d697e82d-cb99...,19089769,...,4.95,4.90,4.85,M130008732,f,2,2,0,0,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13717,1052523990478131414,https://www.airbnb.com/rooms/1052523990478131414,20231227173752,2023-12-28,city scrape,Rental unit in Katsushika City · ★New · 2 bedr...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,552216877,...,NaN,NaN,NaN,M130035708,t,2,2,0,0,NaN
13718,1052570322244321995,https://www.airbnb.com/rooms/1052570322244321995,20231227173752,2023-12-28,city scrape,Home in Shinagawa City · ★New · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,28621978,...,NaN,NaN,NaN,M130025425,f,7,0,7,0,NaN
13719,1052573121521757761,https://www.airbnb.com/rooms/1052573121521757761,20231227173752,2023-12-28,city scrape,Home in Shinagawa City · ★New · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,28621978,...,NaN,NaN,NaN,M130025425,f,7,0,7,0,NaN
13721,1052618956131641134,https://www.airbnb.com/rooms/1052618956131641134,20231227173752,2023-12-28,city scrape,Rental unit in Taito City · ★New · 2 bedrooms ...,NaN,Ichiyu Tokyo Apartment is near Iriya and Asaku...,https://a0.muscache.com/pictures/miso/Hosting-...,292115979,...,NaN,NaN,NaN,M130019957,f,47,45,2,0,NaN


In [9]:
data.room_type.value_counts()

room_type
Entire home/apt    7931
Private room       2276
Name: count, dtype: int64